#	What is Corrective-RAG?

Corrective RAG is a technique that introduces an additional step to verify and correct the information retrieved before generating the final response. This method aims to reduce errors and inconsistencies in the generated output by cross-checking the retrieved information against known facts or trusted sources. It often involves a separate model or module dedicated to fact-checking and error correction.

# Corrective RAG Implementation

1. **Initial Retrieval**: We retrieve the top 3 most relevant documents for the query.
2. **Initial Response Generation**: Using the retrieved context, we generate an initial response.
3. **Critique Generation**: We ask the model to critique its own response, identifying potential errors or missing information.
4. **Additional Retrieval**: Based on the critique, we retrieve additional relevant documents.
5. **Final Response Generation**: We generate an improved response considering the initial response, critique, and additional context.

# Install required libraries

In [1]:
!pip install -q -U \
     Sentence-transformers==3.0.1 \
     langchain==0.2.11 \
     langchain-google-genai==1.0.7 \
     langchain-chroma==0.1.2 \
     langchain-community==0.2.10 \
     einops==0.8.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.3/990.3 kB 37.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 86.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.3/584.3 kB 46.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 87.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 379.9/379.9 kB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.1/140.1 kB 13.7 MB/s et

# Import related libraries related to Langchain, HuggingfaceEmbedding

In [15]:
from langchain_google_genai import (
    ChatGoogleGenerativeAI,
    HarmBlockThreshold,
    HarmCategory,
)
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema import Document
from langchain.document_loaders import WebBaseLoader

In [4]:
import os
import getpass

# Provide Google API Key. You can create Google API key at following link

[Google Gemini-Pro API Creation Link](https://console.cloud.google.com/apis/credentials)

[YouTube Video](https://www.youtube.com/watch?v=ZHX7zxvDfoc)



In [5]:
os.environ["GOOGLE_API_KEY"] = getpass.getpass()

··········


# Provide Huggingface API Key. You can create Huggingface API key at following link

[Higgingface API Creation Link](https://huggingface.co/settings/tokens)




In [6]:
os.environ["HF_TOKEN"] = getpass.getpass()

··········


# Step 1: Load and preprocess data code

In [16]:
def load_and_process_data(url):
    # Load data from web
    loader = WebBaseLoader(url)
    data = loader.load()

    # Split text into chunks (Experiment with Chunk Size and Chunk Overlap to get optimal chunking)
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    chunks = text_splitter.split_documents(data)

    return chunks

# Step 2: Create vector store code

In [17]:
def create_vector_store(chunks):
    embeddings = HuggingFaceEmbeddings()
    vectorstore = Chroma.from_documents(chunks, embeddings)
    return vectorstore

# Step 3: Corrective RAG related code

1. **Initial Retrieval:** We retrieve the top 3 most relevant documents for the query.
2. **Initial Response Generation:** Using the retrieved context, we generate an initial response.
3. **Critique Generation:** We ask the model to critique its own response, identifying potential errors or missing information.
4. **Additional Retrieval:** Based on the critique, we retrieve additional relevant documents.
5. **Final Response Generation:** We generate an improved response considering the initial response, critique, and additional context.

In [18]:
def corrective_rag(query, vectorstore, llm):
    # Initial retrieval
    initial_docs = vectorstore.similarity_search(query, k=3)
    initial_context = "\n".join([doc.page_content for doc in initial_docs])

    # Generate initial response
    initial_prompt = ChatPromptTemplate.from_template(
        "Based on the following context, please answer the query:\nContext: {context}\nQuery: {query}"
    )
    initial_chain = initial_prompt | llm
    initial_response = initial_chain.invoke({"context": initial_context, "query": query})

    # Generate critique
    critique_prompt = ChatPromptTemplate.from_template(
        "Please critique the following response to the query. Identify any potential errors or missing information:\nQuery: {query}\nResponse: {response}"
    )
    critique_chain = critique_prompt | llm
    critique = critique_chain.invoke({"response": initial_response.content, "query": query})

    # Retrieve additional information based on critique
    additional_docs = vectorstore.similarity_search(critique.content, k=2)
    additional_context = "\n".join([doc.page_content for doc in additional_docs])

    # Generate final response
    final_prompt = ChatPromptTemplate.from_template(
        "Based on the initial response, critique, and additional context, please provide an improved answer to the query:\nInitial Response: {initial_response}\nCritique: {critique}\nAdditional Context: {additional_context}\nQuery: {query}"
    )
    final_chain = final_prompt | llm
    final_response = final_chain.invoke({
        "initial_response": initial_response.content,
        "critique": critique.content,
        "additional_context": additional_context,
        "query": query
    })

    return final_response.content

# Step 4: Create chunk of web data to Chroma Vector Store

In [22]:
# Initialize the gemini-pro language model with specified settings (Change temeprature  and other parameters as per your requirement)
llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro-latest", temperature=0.3, safety_settings={
          HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
        },)

# Load and process data
url = "https://en.wikipedia.org/wiki/Artificial_intelligence"
chunks = load_and_process_data(url)

# Create vector store
vectorstore = create_vector_store(chunks)

# Step 5: Run Corrective RAG

This implementation demonstrates the key aspects of Corrective RAG:

1. Initial retrieval and response generation
2. Self-critique to identify potential improvements
3. Additional retrieval based on the critique
4. Final response generation incorporating all available information

In [23]:
# Example query
query = "What are the main applications of artificial intelligence?"

response = corrective_rag(query, vectorstore, llm)

print("Final Response:")
print(response)

Final Response:
## The Main Applications of Artificial Intelligence

Artificial intelligence is rapidly changing the world around us, with applications spanning across numerous fields. Here are some of the most impactful areas where AI is making a difference:

**1. Healthcare:** AI is revolutionizing healthcare with applications like:

* **Medical Image Analysis:** AI algorithms can analyze medical images like X-rays, CT scans, and MRIs to detect diseases like cancer, pneumonia, and even eye conditions with remarkable accuracy, often surpassing human experts.
* **Drug Discovery:** AI accelerates the drug discovery process by analyzing vast datasets of molecular structures and biological information, identifying potential drug candidates and predicting their effectiveness.
* **Personalized Treatment:** AI helps tailor treatment plans to individual patients by analyzing their medical history, genetic information, and lifestyle factors, leading to more effective and personalized healthcar